In [ ]:
#Installing Dependencies
# %%
!pip install transformers torch # Installing torch and transformers

import torch # Importing torch module for using its functionalities
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification




In [ ]:
#Defining Text and Labels
texts = [
    "This is an example sentence.",
    "Each sentence is a separate text.",
    "We will process them using BERT.",
    "BERT is a powerful language model.",
    "It can perform various NLP tasks.",
    "Such as sentiment analysis, text classification.",
    "And question answering.",
    "We will use a pre-trained BERT model.",
    "And fine-tune it on this small dataset.",
    "This is the last example sentence."]

In [ ]:
labels = [0, 1, 0, 1, 0, 1, 0, 1, 0, 1] # Example labels, replace with your actual labels


In [ ]:
#Displaying Text and Label Pairs
for idx, text in enumerate(texts):
    print(f"Text {idx + 1}: {labels[idx]}")

Text 1: 0
Text 2: 1
Text 3: 0
Text 4: 1
Text 5: 0
Text 6: 1
Text 7: 0
Text 8: 1
Text 9: 0
Text 10: 1


In [ ]:
#Hyperparameters Setup
BATCH_SIZE=2
EPOCHS=3
LEARNING_RATE=2e-5

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#Loading the Pre-Trained BERT Model
model_name="bert-base-uncased"
tokenizer=BertTokenizer.from_pretrained(model_name)
model=BertForSequenceClassification.from_pretrained(model_name,num_labels=2).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
Custom Dataset Class
class SequentialDataset(torch.utils.data.Dataset):
  def __init__(self,texts,labels,tokenizer):
    self.texts=texts
    self.labels=labels
    self.tokenizer=tokenizer

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )

        input_ids = encoded_inputs['input_ids'].squeeze()
        attention_mask = encoded_inputs['attention_mask'].squeeze()
        label = torch.tensor(label, dtype=torch.long)

        return input_ids, attention_mask, label


In [ ]:
#Dataset and DataLoader

dataset=SequentialDataset(texts,labels,tokenizer)
dataloader=DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True)

In [ ]:
#Optimizer Setup
optimizer=torch.optim.AdamW(model.parameters(),lr=LEARNING_RATE)

In [ ]:
#Training Loop
print("--------------")
for epoch in range(EPOCHS):
  print(f"Epochs {epoch+1}/{EPOCHS}")
  for batch in dataloader:
    input_ids,attention_mask,labels=batch
    total_loss=0
    for batch in dataloader:
      input_ids,attention_mask,labels=batch
      attention_mask=attention_mask.to(device)
      labels=labels.to(device)

      outputs = model(input_ids, attention_mask=attention_mask,labels=labels)
      loss=outputs.loss
      logits=outputs.logits

      total_loss+=loss.item()

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    print(f"Total Loss: {total_loss}/{len(dataloader):4f}")

--------------
Epochs 1/3
Total Loss: 3.5801185965538025/5.000000
Total Loss: 3.015259563922882/5.000000
Total Loss: 2.63053560256958/5.000000
Total Loss: 2.1095229983329773/5.000000
Total Loss: 1.6980415284633636/5.000000
Epochs 2/3
Total Loss: 1.2982762455940247/5.000000
Total Loss: 0.8789935559034348/5.000000
Total Loss: 0.5940317958593369/5.000000
Total Loss: 0.39498552680015564/5.000000
Total Loss: 0.26388878002762794/5.000000
Epochs 3/3
Total Loss: 0.18752576410770416/5.000000
Total Loss: 0.13051317632198334/5.000000
Total Loss: 0.10121140070259571/5.000000
Total Loss: 0.07724734023213387/5.000000
Total Loss: 0.060354938730597496/5.000000


In [ ]:
#Evaluating the Model
model.eval()
test_texts = [
    "BERT is a powerful language model.",
    "It can perform various NLP tasks.",
    "Such as sentiment analysis, text classification.",
    "And question answering.",
    "We will use a pre-trained BERT model."]

test_labels=[0,1,0,1,0,]

test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')

In [ ]:
input_ids = test_encodings['input_ids'].to(device)
attention_mask = test_encodings['attention_mask'].to(device)

In [ ]:
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

In [ ]:
#Accuracy Calculation

print("\n-------------------------------------------")
print('Testing those comments: ')

!pip install scikit-learn # install the necessary package for accuracy_score
from sklearn.metrics import accuracy_score

for idx, text in enumerate(test_texts):
  print(f"Text {idx + 1}: {test_labels[idx]}")
accuracy=accuracy_score(test_labels,predictions)
print(f'prediction: {predictions}')
print(f"labels: {test_labels}")


-------------------------------------------
Testing those comments: 
Text 1: 0
Text 2: 1
Text 3: 0
Text 4: 1
Text 5: 0
prediction: tensor([1, 0, 1, 0, 1])
labels: [0, 1, 0, 1, 0]
